In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
openai_model = os.getenv("OPENAI_MODEL_NAME")
os.environ["OPENAI_MODEL_NAME"]= openai_model
os.environ["OPENAI_API_KEY"]= openai_key

question = "What is the political landscape gainesville, fl?"

In [2]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

template = """You are an AI language model assistant. Your task is to generate 5 different versions of
    the given user questions to retrieve relevant data from the internet. In order to this well, you will need to 
    think like a political consultant incharge of running a campaign. By generating multiple perspectives on the user question, 
    your goal is to help the user get a better answer with a full picture. Provide these alternative questions separated by newlines. 
    Original question: {question}"""
decomp_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
query_analyzer = (decomp_prompt | llm | StrOutputParser() | (lambda x: x.split("\n")))
sub_queries = query_analyzer.invoke({"question":question})
sub_queries

['1. How do political parties in Gainesville, FL influence local policies and decision-making?',
 '2. What are the key political issues currently shaping the landscape in Gainesville, FL?',
 '3. How do demographic changes impact the political dynamics in Gainesville, FL?',
 '4. What role do local interest groups play in shaping the political landscape of Gainesville, FL?',
 '5. How has the political landscape of Gainesville, FL evolved over the past decade?']

## Define Agents

In [3]:
# import crewai

# define an agent
    # duckduckgo
    # wikipedia agent
    # web scraper focused on newspapers

# for loop
# each query 
# crew = Crew(
#   agents=[researcher, manager],
#   tasks=[task1, task2],
#   process=Process.sequential,
#   verbose=2
# )
# result = crew.kickoff() 

In [4]:
from crewai import Agent, Task, Crew, Process
from crewai.task import TaskOutput
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
# from crewai_tools import (
#     WebsiteSearchTool,
#     ScrapeWebsiteTool
# )
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool,
    ScrapeWebsiteTool
)

search_tool = DuckDuckGoSearchRun()
# wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
# wikidata_tool = WikidataQueryRun(api_wrapper=WikidataAPIWrapper())
website_search_tool = WebsiteSearchTool()
scrape_tool = ScrapeWebsiteTool()

In [5]:
# Create agents
search_agent = Agent(
    role='Researcher',
    goal='Find up-to-date information and news articles',
    backstory="""
    You are a research expert. 
    Your goal is to provide answers based on information from the internet. 
    You must use the provided tools to find relevant online information. 
    You should never use your own knowledge to answer questions.
    It is mandatory to include the sources of your information in the report.
    """,
    tools=[search_tool, website_search_tool],
    # verbose=True
)

In [6]:
# Create agents
scraper_agent = Agent(
    role='Scraper',
    goal='Find the most relevant websites related to search query.',
    backstory="""
    You are a web scraping expert. 
    Your goal is to provide answers based on information from the internet. 
    You must use the provided tools to find relevant online information. 
    You should never use your own knowledge to answer questions.
    It is mandatory to include the sources of your information in the report.
    """,
    tools=[scrape_tool],
    # verbose=True
)

In [7]:
# Create agents
campaign_manager = Agent(
    role='Political Campaign Manager & Report Writer',
    goal='Analyze the provided research data to generate a comprehensive report that summarizes key insights, draws conclusions, and provides strategic recommendations to inform the campaign strategy.',
    backstory="""
    You are an experienced Political Campaign Manager with exceptional analytical and writing skills.
    Your role is to thoroughly review all the research data gathered by the team, identify the most important trends and insights, 
    and synthesize the information into a clear, compelling report. Your report should provide a high-level overview of the political
    landscape, highlight key opportunities and risks for the campaign, and offer specific recommendations on messaging, 
    voter targeting, and resource allocation. Your ability to distill complex data into an actionable narrative will be critical
    to aligning the team and guiding campaign decisions.
    It is mandatory to include the sources of your information in the report.
    """,
    tools=[scrape_tool],
    # verbose=True
)

In [8]:
query_task_list = []

for query in sub_queries:
    # iterate through queries, define search and scrape task
    search_task = Task(
        description=f"""Search the web to gather information on {query} using trusted sources such as 
        """,
        expected_output="A list of facts",
        agent= search_agent,
        tools=[search_tool, website_search_tool]
    )

    scrape_task = Task(
        description="""Take the data provided and write a detailed report to help an election campaign come up with a plan to win.  
            """,
        agent=scraper_agent,
        expected_output="A report on the city",
        tools=[scrape_tool],
    )

# once tasks are defined for each query, append to task_list
query_task_list.append(search_task)
# query_task_list.append(scrape_task)


In [9]:
# Create final task for summary/report generation
report_task = Task(
    description="""
    Take the data provided and write a detailed report to help our election campaign.  
    """,
    agent=campaign_manager,
    # report structure in valid json format
    expected_output="""
    Agent: Political Campaign Manager & Report Writer
    Goal: Analyze the provided research data to generate a comprehensive report that summarizes key insights, draws conclusions, and provides strategic recommendations to inform the campaign strategy.
    """,
    tools=[],
    verbose=True
)
query_task_list.append(report_task)



In [10]:
# create the crew
# Instantiate crew with a sequential process
crew = Crew(
  agents=[search_agent, scraper_agent, campaign_manager], # add in the agents
  tasks=query_task_list, #add in the tasks 
  process=Process.sequential,
  verbose=2
)
result = crew.kickoff()

 [DEBUG]: == Working Agent: Researcher
 [INFO]: == Starting Task: Search the web to gather information on 5. How has the political landscape of Gainesville, FL evolved over the past decade? using trusted sources such as 
        
 

In conclusion, the political landscape of 1950 was complex and multifaceted, with competing interests and ideologies shaping the course of history. The post-war years of 1950s America were a period of both power and prosperity, but they were also marked by challenges and conflicts that would have far-reaching implications for the decades to come. The 2020 presidential election exemplified Florida's political complexities. Although Republican candidate Donald Trump emerged victorious, the margin of victory was narrow. This close race emphasized the divided nature of the state's electorate and the rising influence of urban areas, where Democratic support is particularly strong. A look at Prince William's politics over the past decade: Blue with pockets of pur

Steps:
1. Get query
2. Expand Query using Decomposition
3. Pass all sub-queries to crew ai agents
4. Review all info
5. create a memo

Agents
1. scraper
2. search duck duck go
3. Compiler/Reviewer - request more info from agents 1 and 2?
4. Memo Writer

## Decomposition Step

steps for the morning:
- get up out of bed.
- brush up on my teeth, i mean brush up on minimax algorythm and alpha beta pruning.
- get addressed. i mean address the prompts that param asked me to fix.

# production RAG: context is all you need.
production RAG employs various strategies in query translation, routing, query construction, retrieval, generation to provide the LLM with the best possible context to answer the query.


# ![test](rag-from-scratch-flowchart.png)
source: 


# ![General Query Transformation](general-query-transformation.jpeg)


# ![General Query Transformation Result Intuition](general-query-transformation-result-intuition.jpeg)